<a href="https://colab.research.google.com/github/Harsharya3107/Automatic-Music-Transcription/blob/main/CNN_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras import Input
import numpy as np 
import pandas as pd 
import os

from keras.layers import Conv2D, MaxPool2D, add,Dropout,Dense,Flatten
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
mini_batch_size = 512
lr=0.1

In [ ]:
fwavpath ='/content/drive/MyDrive/train/wav'
train_files = [f1 for f1 in os.listdir(fwavpath)]
num_batches =  len(train_files)
print(num_batches)
# Y_t = np.array(np.load(os.path.join(fmidpath,ytrain_files[i])))

for i in range((int)(num_batches)):
  
  X_t = np.array(np.load(os.path.join(fwavpath,train_files[i])))
  if(np.size(X_t,1)<252):
    num_missing = 252-np.size(X_t,1)
    X_t = np.pad(X_t,(0,num_missing))
  if i==0:
    X=X_t
  else:
    X=np.concatenate((X,X_t), axis = 0)
  
print(X.shape)

180
(3728268, 252)


In [ ]:
fmidpath ='/content/drive/MyDrive/train/mid'
ytrain_files = [f1 for f1 in os.listdir(fmidpath)]
num_batches =  len(ytrain_files)
print(num_batches)
# Y_t = np.array(np.load(os.path.join(fmidpath,ytrain_files[i])))

for i in range((int)(num_batches)):
  
  Y_t = np.array(np.load(os.path.join(fmidpath,ytrain_files[i])))
  if(np.size(Y_t,1)<88):
    num_missing = 88-np.size(Y_t,1)
    Y_t = np.pad(Y_t,(0,num_missing))
  if i==0:
    Y=Y_t
  else:
    Y=np.concatenate((Y,Y_t), axis = 0)
  
print(Y.shape)

180
(3728268, 88)


In [ ]:
# for i in range(int(num_batches)):

#   if train_files[i].find("CQT")!=-1:
#     X_t = np.array(np.load(os.path.join(fpath,train_files[i])))
    
#   if train_files[i].find("label")!=-1 :
#     Y_t = np.array(np.load(os.path.join(fpath,train_files[i])))
    
#   if i==0:
#     X= X_t
#     Y= Y_t
#   else:
#     X=np.concatenate((X,X_t), axis = 0)
#     Y=np.concatenate((Y,Y_t), axis = 0)
# print(X.shape)
# print(Y.shape)

In [ ]:
X=X[:Y.shape[0],:]
print(X.shape)

(3728268, 252)


In [ ]:
from sklearn.model_selection import train_test_split
test_size = 0.2
validation_size = 0.2
#train test split
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=test_size)
    
# create train validation split
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

In [ ]:
X_train = X_train[:int(X_train.shape[0]/mini_batch_size)*mini_batch_size,:]
print(X_train.shape)

(2385920, 252)


In [ ]:
y_train = y_train[:int(y_train.shape[0]/mini_batch_size)*mini_batch_size,:]
print(y_train.shape)

(2385920, 88)


In [ ]:
X_train=X_train.reshape(list(X_train.shape) + [1])
X_train=X_train.reshape(list(X_train.shape) + [1])

In [ ]:
X_validation=X_validation.reshape(list(X_validation.shape) + [1])
X_validation=X_validation.reshape(list(X_validation.shape) + [1])

In [ ]:
print(X_train.shape)
print(y_train.shape)

(2385920, 252, 1, 1)
(2385920, 88)


In [ ]:
y_train_bool = np.bool_(y_train)
mini_batch_size = 8
minibatch= True

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((X_train,y_train_bool))
train_data = train_data.batch(mini_batch_size)

validation_data = tf.data.Dataset.from_tensor_slices((X_validation,y_validation))
validation_data = validation_data.batch(mini_batch_size)

In [ ]:
print(train_data)

<BatchDataset shapes: ((None, 252, 1, 1), (None, 88)), types: (tf.float32, tf.bool)>


In [ ]:
window_size=7
min_midi = 21
max_midi = 108
note_range = max_midi - min_midi + 1


In [ ]:
input_shape=(int(X_train.shape[0]/mini_batch_size),88,1,88) #number of samples
model=Sequential()
#stage1
model.add(Conv2D(filters=32, kernel_size=(5,5),padding="same", activation='relu', input_shape=input_shape))
model.add(BatchNormalization())
model.add(Dropout(rate=0.2))

#stage2
model.add(Conv2D(filters=32, kernel_size=(3, 3),padding="same", activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(rate=0.2))
model.add(MaxPool2D(pool_size=(2, 2),padding="same"))

model.add(Flatten())
#stage3
model.add(Dense(1000  , activation ='relu'))
model.add(BatchNormalization())
model.add(Dropout(rate=0.5))

#stage4
model.add(Dense(200, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(rate=0.5))

#stage5
model.add(Dense(88, activation='softmax'))

In [ ]:
# input_shape=(int(X_train.shape[0]/400),88,1,88) #number of samples
# model=Sequential()
# #stage1
# model.add(Conv2D(filters=64, kernel_size=(5,4),padding="same", activation='relu', input_shape=input_shape))
# model.add(BatchNormalization())
# model.add(Dropout(rate=0.2))

# #stage2
# model.add(Conv2D(filters=64, kernel_size=(3, 3),padding="same", activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(rate=0.2))
# model.add(MaxPool2D(pool_size=(1, 2),padding="same"))

# model.add(Flatten())
# #stage3
# model.add(Dense(1024 , activation ='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(rate=0.5))

# #stage4
# model.add(Dense(512, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(rate=0.5))

# #stage5
# model.add(Dense(88, activation='sigmoid'))

In [ ]:
optimizer1 = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer= optimizer1, metrics=['accuracy'])
model.fit(X_train,y_train,epochs=30,batch_size=mini_batch_size,validation_data=validation_data, verbose = 1)

Epoch 1/30
298240/298240 [==============================] - 1246s 4ms/step - loss: 0.0837 - accuracy: 0.3247 - val_loss: 0.0794 - val_accuracy: 0.3403
Epoch 2/30
298240/298240 [==============================] - 1208s 4ms/step - loss: 0.0753 - accuracy: 0.3424 - val_loss: 0.0978 - val_accuracy: 0.3475
Epoch 3/30
298240/298240 [==============================] - 1218s 4ms/step - loss: 0.0734 - accuracy: 0.3452 - val_loss: 0.0544 - val_accuracy: 0.3922
Epoch 4/30
298240/298240 [==============================] - 1205s 4ms/step - loss: 0.0723 - accuracy: 0.3462 - val_loss: 0.0825 - val_accuracy: 0.3633
Epoch 5/30
298240/298240 [==============================] - 1200s 4ms/step - loss: 0.0715 - accuracy: 0.3478 - val_loss: 0.0702 - val_accuracy: 0.3821
Epoch 6/30
298240/298240 [==============================] - 1199s 4ms/step - loss: 0.0708 - accuracy: 0.3485 - val_loss: 0.0591 - val_accuracy: 0.3877
Epoch 7/30
298240/298240 [==============================] - 1186s 4ms/step - loss: 0.0704 - ac

KeyboardInterrupt: ignored

In [1]:
model.summary()

NameError: ignored